In [1]:
## IMPORTAMOS LAS LIBRERÍAS NECESARIAS  
import os
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook 

## creación de variable con los datos de las celdas que se compararan

In [2]:
rangos = pd.read_csv(r'W:\2023\FINANZAS\AI-2023\Codigos NRIP\Codigos\validador de estructura\rangos1.csv')
rangos['Rangos'] = rangos['Rangos'].str.split(',')
# Rellenar con ceros a la izquierda en la columna 'clave' para tener una longitud de 3 caracteres
rangos['ID'] = rangos['ID'].apply(lambda x: str(x).zfill(3))

# Luego, agrupamos los valores por la columna 'clave' y los convertimos en listas
rangos = rangos.groupby('ID')['Rangos'].apply(lambda x: [item for sublist in x for item in sublist]).to_dict()

In [3]:
def compara_estructura(carpeta, carpeta1):
    archivos_con_discrepancias = []  # Lista para almacenar los nombres de archivos con discrepancias
    archivos = os.listdir(carpeta)
    for i in os.listdir(carpeta1):
        for archivo in archivos:
            if i[:3] == archivo[:3]:
                ruta1 = os.path.join(carpeta, archivo)
                ruta2 = os.path.join(carpeta1, i)
                try:
                    wb = load_workbook(ruta1)
                    wb2 = load_workbook(ruta2)
                    wb = wb.active
                    wb2 = wb2.active
                    data = []
                    for datos in rangos[f'{i[:3]}']:
                        if wb[f'{datos}'].value != wb2[f'{datos}'].value:
                            data.append(datos)
                        else:
                            pass
                except:
                    print(i, "NO SE PUDO PROCESAR")
                if len(data)>0:
                    archivos_con_discrepancias.append(f'en {i} no coinciden {data}')

    if len(archivos_con_discrepancias)>0:
        ruta = input("INSERTE LA RUTA DONDE QUIERE GUARDAR EL ARCHIVO CON ERRORES")
        rutaentera = os.path.join(ruta,f'{os.path.basename(carpeta1)}.txt')
        with open(rutaentera, "w") as archivo_txt:
            for elemento in archivos_con_discrepancias:
                archivo_txt.write(elemento + "\n")
    else:
        print(f'SIN ERRORES EN LA EMPRESA {os.path.basename(carpeta1)}')
        # Si hay archivos con discrepancias, imprime el mensaje final con los nombres de los archivos
        #archivos_str = ", ".join(archivos_con_discrepancias)
        #print(f"En la empresa {os.path.basename(carpeta1)} los archivos: {archivos_str} fueron enviados con errores")
    # Llamar a la función con las carpetas adecuadas

# el codigo me dice si la empresa tiene archivos con errores o no.

In [6]:
carpeta = r'W:\2023\FINANZAS\AI-2023\formatos_verificar' #ruta donde se encuentran los formatos de la norma 043
carpeta1 = input("inserte donde se encuentran los archivos de la empresa")
compara_estructura(carpeta, carpeta1)